# **Projeto AM 2019.1 - K-medoid-fuzzy-cluster**

***Parte 1 - Implementação e Execução do Algoritmo MVFCMddV***

Importação das bibliotecas usadas no projeto

In [1]:
from scipy.spatial.distance import pdist, squareform
from scipy import stats
from scipy.stats import wilcoxon
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold, StratifiedKFold, cross_val_score, RepeatedStratifiedKFold, train_test_split, StratifiedShuffleSplit
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import GaussianNB
from numpy.linalg import inv, det
from statistics import mean
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import adjusted_rand_score
import math
import time

Extração dos dados de csv para matrizes em Python e normalização dos dados

In [4]:
def ExtractData (filename):

    features = []
    classes = []
    
    file = open(filename, 'r')

    for i in range(10):
        classes.append([i]*200)

    for line in file:
        row = line.split(';')
        features.append([float(x) for x in row ])

    mat_feat = np.matrix(features).astype(np.float)

    #matriz de dados
    return (mat_feat)

mat_fac = ExtractData('mfeat-fac.csv').astype(int)
mat_fou = ExtractData('mfeat-fou.csv')
mat_kar = ExtractData('mfeat-kar.csv')

#normalização dos dados

std = StandardScaler()

fac = std.fit_transform(mat_fac)
fou = std.fit_transform(mat_fou)
kar = std.fit_transform(mat_kar)


Cálculo das dissimilaridades usando a distância euclidiana

In [5]:
#calculo das matrizes de dissimilaridade
from scipy.spatial.distance import pdist, squareform


#dados não normalizados:
#distancias_fac = pdist(mat_fac, metric='euclidean')
#distancias_fou = pdist(mat_fou, metric='euclidean')
#distancias_kar = pdist(mat_kar, metric='euclidean')

#considerando os dados normalizados

distancias_fac = pdist(fac, metric='euclidean')
distancias_fou = pdist(fou, metric='euclidean')
distancias_kar = pdist(kar, metric='euclidean')

mfeat_fac = squareform(distancias_fac)
mfeat_fou = squareform(distancias_fou)
mfeat_kar = squareform(distancias_kar)


Funções das etapas de inicialização

In [6]:
def initialize_medoid_vector(G):
  iterator = np.nditer(G, op_flags=['readwrite'],flags=['multi_index'])
  while not iterator.finished:
      G[iterator.multi_index] = rdm.randint(2000)
      iterator.iternext()
  return G

In [7]:
def initialize_relevanceWeights(K,p):
  
  return np.ones((K,p))

Funções das etapas de atualização

In [8]:
def update_medoid_vector(G,D,U,n):
  iterator = np.nditer(G,op_flags=['readwrite'],flags=['multi_index'])

  while not iterator.finished:
      k = iterator.multi_index[0]
      dissimilarities_sum = np.zeros([n])
      dissimilarity_matrix = D[iterator.multi_index[1]]
      
      for h in range(n):
        medoid_sum = 0
        medoid_sum += sum((U[:,k]**m)*(dissimilarity_matrix[:,h]))
        dissimilarities_sum[h]=medoid_sum
      G[iterator.multi_index]=(np.argmin(dissimilarities_sum))

      iterator.iternext()

In [9]:
def update_fuzzy_partition(U,G,D,relevance_weights):
  UI = []
  iterator = np.nditer(U,op_flags=['readwrite'],flags=['multi_index'])

  while not iterator.finished:
      dissimilarity_to_k = 0
      for j in range(p):
        dissimilarity_matrix = D[j]
        i = iterator.multi_index[0]
        k = iterator.multi_index[1]

        medoidIndex = int(G[k][j])

        dissimilarity_to_k += (relevance_weights[k,j]*dissimilarity_matrix[i,medoidIndex])
      u = 0
      for h in range(K):
        dissimilarities_to_h = 0  
        for j in range(p):
          dissimilarity_matrix = D[j]

          medoidIndex = int(G[h][j])
          dissimilarities_to_h += (relevance_weights[h,j] *dissimilarity_matrix[i,medoidIndex])


        u += ( dissimilarity_to_k / float(dissimilarities_to_h))**( 1/(m-1)) 

      U[iterator.multi_index] = u**(-1)
      UI=[]
      iterator.iternext()

In [10]:
def update_relevance_weights(relevance_weights,G,U,D):
  
  iterator = np.nditer(relevance_weights, op_flags=['readwrite'],flags=['multi_index'])
  
  while not iterator.finished:
      k = iterator.multi_index[0]
      j = iterator.multi_index[1]
      multiplicator = 1
      for h in range(p):
        weighted_distance_sum = 0
        dissimilarity_matrix = D[h]
        medoid_index = int(G[k][h])
        weighted_distance_sum += sum((U[:,k]**m)*(dissimilarity_matrix[:,medoid_index]))

        multiplicator *= weighted_distance_sum

      distance_sum = 0
      dissimilarity_matrix = D[j]
      medoid_index = int(G[k][j])
      distance_sum += sum((U[:,k]**m)*(dissimilarity_matrix[:,medoid_index]))
      lambda_kj = ((multiplicator**(1/p))/distance_sum)
      relevance_weights[k,j]=lambda_kj
      iterator.iternext()
      

Calculo do valor da Função Objetivo

In [11]:
def adequacy_criteion(G,U,D,m,relevance_weights,p):
  J = 0
  for k in range (K):
    
    for i in range(n):
      
      weighted_distance = 0
      for j in range(p):
        dissimilarity_matrix = D[j]
        medoid_index = int(G[k][j])
        weighted_distance += (relevance_weights[k,j])* dissimilarity_matrix[i,medoid_index] 
      J += (U[i,k]**m) * weighted_distance
        
  return J

Função para checar se foram gerados 10 grupos

In [12]:
def check_empty_cluster(U):
  
  count = np.argmax(U, axis=1)
  
  if(len(list(set(count))) == 10):
    return True
  else:
    return False


Algoritmo de Agrupamento MVFCMddV 

Usa como entrada:
*   D - lista com p matrizes de dissimilaridades com n exemplos
*   m - taxa de aprendizagem
*   k - número de clusters
*   epsilon - critério de convergência
*   T - número máximo de iterações

Retorna como saída:
*   U - matriz de pertencimento
*   W - matriz de pesos de relevância
*   G - matriz de medoids
*   J_Iteractions - lista com os valores das funções objetivos das iterações
*   J - valor da função objetivo final



In [13]:
def fuzzy_clustering():
  # Configuração inicial do algoritmo

  D = [mfeat_fac, mfeat_fou, mfeat_kar]
  n = mat_fac.shape[0]
  m = 1.6 
  K = 10
  p = len(D)

  # Configuração inicial do algoritmo

  # U -> Matriz nxk Grau de pertencimento
  # G -> Vetor de vetores metoids
  # Lambda -> Vetor de vetores de relevância de peso

  # Inicialização 
  # Nível de fuzzyficação
  G = np.zeros((K,p))
  U = np.zeros((n,K))
  T = 150
  epsilon = 10**-10

  # Inicialização das matrizes G, relevance_weights (Lambda) e U75563
  initialize_medoid_vector(G)
  relevance_weights = initialize_relevanceWeights(K,p)
  update_fuzzy_partition(U,G,D,relevance_weights)
  J = adequacy_criteion(G,U,D,m,relevance_weights,p)
  # Começar as iterações do algoritmo
  t=0
  
  best_local_J = J
  best__local_G = np.zeros((K,p))
  best_local_U = np.zeros((n,K))
  best_local_relevance_weights = initialize_relevanceWeights(1,3)
  
  J_values = [J]
  
  while(t < T):
    #print(J)
    t += 1
    update_medoid_vector(G,D,U,n)
    update_relevance_weights(relevance_weights,G,U,D)
    update_fuzzy_partition(U,G,D,relevance_weights)
    old_J = J
    J = adequacy_criteion(G,U,D,m,relevance_weights,p)
    
    J_values.append(J)
    
    if(J < best_local_J):
      best_local_G = G
      best_local_U = U
      best_local_relevance_weights = relevance_weights
      best_local_J = J  
    #print(best_local_J)
    if(abs(old_J - J) < epsilon):
      break
  return({"J": best_local_J, "G": best_local_G, "U": best_local_U, "W": best_local_relevance_weights, "J_Iteractions": J_values})

Função para calcular o Rand Score

In [14]:
def calculate_rand_score(U):
  Y_crisp = np.argmax(U, axis=1)
  true_labels =  np.arange(10*200) // 200
  return adjusted_rand_score(true_labels,Y_crisp)

Célula com algoritmo para executar o algoritmo de agrupamento 100 vezes e armazenar a saída

In [15]:
D = [mfeat_fac, mfeat_fou, mfeat_kar]
p = len(D)
n = mat_fac.shape[0]
m = 1.6 
K = 10
rdm = np.random.RandomState(2)

best_J_list = []
rand_score_list = []
J_Interactions_list = []
times_list = []
u_list = []

best_J = math.inf
it = 0
while(it<100):
  print(it)
  
  start = time.time()
  out = fuzzy_clustering()
  end = time.time()
  
  best_J_list.append(out["J"])
  J_Interactions_list.append(out["J_Iteractions"])
  rand_score_list.append(calculate_rand_score(out["U"]))
  times_list.append(end - start)
  u_list.append(out["U"])
  
  #print("Melhor J:",out["J"])
  if(out["J"] < best_J) and (check_empty_cluster(out["U"])):
    best_G = out["G"]
    best_U = out["U"]
    best_relevance_weights = out["W"]
    best_J = out["J"]
  
  it = it+1  

Y_crisp = np.argmax(best_U, axis=1)

print(best_J)
print(set(Y_crisp))
print(best_J_list)
print(rand_score_list)
print(J_Interactions_list)
print(times_list)

0


KeyboardInterrupt: 